In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import date
import icd10
from icd9 import ICD9
from icd9cms.icd9 import search
import re

In [2]:
import sys
!{sys.executable} -m pip install icd10-cm

In [3]:
import sys
!{sys.executable} -m pip install icd9cms

In [4]:
df = pd.read_excel("./Dataset/contoh data RS.xlsx")

In [5]:
df.head()

,NO,NO. RM,TGL LAHIR,UMUR,SEX,TGL MASUK,TGL KELUAR,CARA KELUAR,CARA BAYAR,LOS,...,TIND15,TIND16,TIND17,TIND18,TIND19,TIND20,ICD KERACUNAN,ICD PENYEBAB MENINGGAL,NAMA KELAS,NAMA INSTALASI
0,1,95797,1962-10-18,77 Thn 11 bln 0 hr,P,30-10-2016 04:45:50,02-11-2016 14:26:18,Pulang Diizinkan,JKN Non PBI,4,...,-,-,-,-,-,-,NaN,NaN,Kelas II (ICU/ICCU),ICCU
1,2,89651,1959-01-30,65 Thn 1 bln 6 hr,L,30-10-2016 16:53:00,12-11-2016 13:22:12,Pulang Diizinkan,Jamkesda Yogyakarta,14,...,-,-,-,-,-,-,NaN,NaN,Kelas III AC,Anggrek 1 (Instalasi Jantung)
2,3,92222,1944-08-10,60 Thn 8 bln 25 hr,L,30-10-2016 18:40:21,02-11-2016 20:11:09,Pulang Diizinkan,JKN Non PBI,4,...,-,-,-,-,-,-,NaN,NaN,Kelas III AC,Anggrek 1 (Instalasi Jantung)
3,4,82586,1943-08-17,71 Thn 8 bln 26 hr,L,31-10-2016 00:14:45,07-12-2016 19:10:00,Pulang Diizinkan,JKN Non PBI,38,...,93.96,-,-,-,-,-,NaN,NaN,Kelas II Non AC,Anggrek 2
4,5,84394,1946-07-22,50 Thn 5 bln 6 hr,P,31-10-2016 02:12:20,04-11-2016 14:29:59,Pulang Diizinkan,JKN Non PBI,5,...,-,-,-,-,-,-,NaN,NaN,Kelas III AC,Anggrek 1 (Instalasi Jantung)


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 97 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   NO                      11 non-null     int64         
 1   NO. RM                  11 non-null     int64         
 2   TGL LAHIR               11 non-null     datetime64[ns]
 3   UMUR                    11 non-null     object        
 4   SEX                     11 non-null     object        
 5   TGL MASUK               11 non-null     object        
 6   TGL KELUAR              11 non-null     object        
 7   CARA KELUAR             11 non-null     object        
 8   CARA BAYAR              11 non-null     object        
 9   LOS                     11 non-null     int64         
 10  ICD UTAMA               11 non-null     object        
 11  K.DIAG.UTAMA            11 non-null     object        
 12  DX1                     11 non-null     object      

In [7]:
df['TGL KELUAR'] = pd.to_datetime(df['TGL KELUAR'], dayfirst=True)
df['TGL MASUK'] = pd.to_datetime(df['TGL MASUK'], dayfirst=True)

In [8]:
df['TGL MASUK'].head()

0   2016-10-30 04:45:50
1   2016-10-30 16:53:00
2   2016-10-30 18:40:21
3   2016-10-31 00:14:45
4   2016-10-31 02:12:20
Name: TGL MASUK, dtype: datetime64[ns]

In [9]:
df['TGL KELUAR'].head()

0   2016-11-02 14:26:18
1   2016-11-12 13:22:12
2   2016-11-02 20:11:09
3   2016-12-07 19:10:00
4   2016-11-04 14:29:59
Name: TGL KELUAR, dtype: datetime64[ns]

In [12]:
df['TGL LAHIR'].head()

0   1962-10-18
1   1959-01-30
2   1944-08-10
3   1943-08-17
4   1946-07-22
Name: TGL LAHIR, dtype: datetime64[ns]

In [10]:
umur = df['TGL MASUK'].sub(df['TGL LAHIR'], axis=0).dt.days.div(365).round().astype(np.int64)
umur.head()

2020-06-29 20:41:51,654: NumExpr defaulting to 8 threads.


0    54
1    58
2    72
3    73
4    70
dtype: int64

In [11]:
dfbaru = pd.DataFrame() 
dfbaru['Usia'] = umur
dfbaru.head()

,Usia
0,54
1,58
2,72
3,73
4,70


In [15]:
LOS = df['TGL KELUAR'].sub(df['TGL MASUK'], axis=0).dt.days
LOS = LOS + 1
LOS

0      4
1     13
2      4
3     38
4      5
5      3
6      3
7      4
8      4
9      8
10    16
dtype: int64

In [16]:
dfbaru['LOS'] = LOS 
dfbaru

,Usia,LOS
0,54,4
1,58,13
2,72,4
3,73,38
4,70,5
5,60,3
6,66,3
7,54,4
8,59,4
9,68,8


In [17]:
code = icd10.find(df["DX1"][0])
print(code.description) 

ST elevation (STEMI) myocardial infarction of anterior wall


In [18]:
dfbaru

,Usia,LOS
0,54,4
1,58,13
2,72,4
3,73,38
4,70,5
5,60,3
6,66,3
7,54,4
8,59,4
9,68,8


In [19]:
a = 1
for x in range(2,32):
    dfbaru.insert(x, "DX"+ str(a), "-")
    a = a+1

In [20]:
dfbaru.head(3)

,Usia,LOS,DX1,DX2,DX3,DX4,DX5,DX6,DX7,DX8,...,DX21,DX22,DX23,DX24,DX25,DX26,DX27,DX28,DX29,DX30
0,54,4,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
1,58,13,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
2,72,4,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-


In [21]:
df.filter(regex=("DX.*"))

,DX1,DX2,DX3,DX4,DX5,DX6,DX7,DX8,DX9,DX10,...,DX21,DX22,DX23,DX24,DX25,DX26,DX27,DX28,DX29,DX30
0,I21.0,I44.0,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
1,I21.0,I44.2,N17.9,E11.7,E11.5,E66.9,E87.1,I10,E78.8,E11.4,...,-,-,-,-,-,-,-,-,-,-
2,I48,I25.1,K71.0,I47.2,Z95.0,E88.0,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
3,I63.9,E11.9,I21.4,J16.8,N17.9,I10,E79.0,E87.5,D64.9,-,...,-,-,-,-,-,-,-,-,-,-
4,I21.0,I10,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
5,I25.2,I20.9,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
6,I20.9,I10,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
7,I22.0,E87.5,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
8,I20.9,I10,J45.9,Y45.1,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
9,I25.2,N17.9,-,-,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-


In [22]:
for x in df.filter(regex=("DX.*")):
    a = 0
    for y in df[x]:
        try:
            code = icd10.find(y)
            dfbaru[x][a] = str(code.description)
            a = a + 1
        except:
            dfbaru[x][a] = str('-')
            a = a + 1

C:\Users\Irvin\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Irvin\anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [23]:
dfbaru

,Usia,LOS,DX1,DX2,DX3,DX4,DX5,DX6,DX7,DX8,...,DX21,DX22,DX23,DX24,DX25,DX26,DX27,DX28,DX29,DX30
0,54,4,ST elevation (STEMI) myocardial infarction of ...,"Atrioventricular block, first degree",-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
1,58,13,ST elevation (STEMI) myocardial infarction of ...,"Atrioventricular block, complete","Acute kidney failure, unspecified",-,Type 2 diabetes mellitus with circulatory comp...,"Obesity, unspecified",Hypo-osmolality and hyponatremia,Essential (primary) hypertension,...,-,-,-,-,-,-,-,-,-,-
2,72,4,Atrial fibrillation and flutter,Atherosclerotic heart disease of native corona...,Toxic liver disease with cholestasis,Ventricular tachycardia,Presence of cardiac pacemaker,"Disorders of plasma-protein metabolism, not el...",-,-,...,-,-,-,-,-,-,-,-,-,-
3,73,38,"Cerebral infarction, unspecified",Type 2 diabetes mellitus without complications,Non-ST elevation (NSTEMI) myocardial infarction,Pneumonia due to other specified infectious or...,"Acute kidney failure, unspecified",Essential (primary) hypertension,Hyperuricemia without signs of inflammatory ar...,Hyperkalemia,...,-,-,-,-,-,-,-,-,-,-
4,70,5,ST elevation (STEMI) myocardial infarction of ...,Essential (primary) hypertension,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
5,60,3,Old myocardial infarction,"Angina pectoris, unspecified",-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
6,66,3,"Angina pectoris, unspecified",Essential (primary) hypertension,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
7,54,4,Subsequent ST elevation (STEMI) myocardial inf...,Hyperkalemia,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
8,59,4,"Angina pectoris, unspecified",Essential (primary) hypertension,Other and unspecified asthma,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
9,68,8,Old myocardial infarction,"Acute kidney failure, unspecified",-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-


In [25]:
for y in df['TIND3']:
    a = str(search(str(y)))
    a= ''.join([i for i in a if not i.isdigit()])
    a=  a.replace(":", "")
    print(a)

None
Bacterial infection in conditions classified elsewhere and of unspecified siteNone
ChilblainsChilblains
BlepharoconjunctivitisNone
None
None
None
Slow virus infection and prion diseases of central nervous systemNone
Bacterial infection in conditions classified elsewhere and of unspecified siteNone
None
None


In [26]:
a = 1
for x in range(32,52):
    dfbaru.insert(x, "TIND"+ str(a), "-")
    a = a+1

In [27]:
dfbaru

,Usia,LOS,DX1,DX2,DX3,DX4,DX5,DX6,DX7,DX8,...,TIND11,TIND12,TIND13,TIND14,TIND15,TIND16,TIND17,TIND18,TIND19,TIND20
0,54,4,ST elevation (STEMI) myocardial infarction of ...,"Atrioventricular block, first degree",-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
1,58,13,ST elevation (STEMI) myocardial infarction of ...,"Atrioventricular block, complete","Acute kidney failure, unspecified",-,Type 2 diabetes mellitus with circulatory comp...,"Obesity, unspecified",Hypo-osmolality and hyponatremia,Essential (primary) hypertension,...,-,-,-,-,-,-,-,-,-,-
2,72,4,Atrial fibrillation and flutter,Atherosclerotic heart disease of native corona...,Toxic liver disease with cholestasis,Ventricular tachycardia,Presence of cardiac pacemaker,"Disorders of plasma-protein metabolism, not el...",-,-,...,-,-,-,-,-,-,-,-,-,-
3,73,38,"Cerebral infarction, unspecified",Type 2 diabetes mellitus without complications,Non-ST elevation (NSTEMI) myocardial infarction,Pneumonia due to other specified infectious or...,"Acute kidney failure, unspecified",Essential (primary) hypertension,Hyperuricemia without signs of inflammatory ar...,Hyperkalemia,...,-,-,-,-,-,-,-,-,-,-
4,70,5,ST elevation (STEMI) myocardial infarction of ...,Essential (primary) hypertension,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
5,60,3,Old myocardial infarction,"Angina pectoris, unspecified",-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
6,66,3,"Angina pectoris, unspecified",Essential (primary) hypertension,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
7,54,4,Subsequent ST elevation (STEMI) myocardial inf...,Hyperkalemia,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
8,59,4,"Angina pectoris, unspecified",Essential (primary) hypertension,Other and unspecified asthma,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
9,68,8,Old myocardial infarction,"Acute kidney failure, unspecified",-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-


In [28]:
df.filter(regex=("TIND.*"))

,TIND1,TIND2,TIND3,TIND4,TIND5,TIND6,TIND7,TIND8,TIND9,TIND10,TIND11,TIND12,TIND13,TIND14,TIND15,TIND16,TIND17,TIND18,TIND19,TIND20
0,0.66,36.07,0.4,0.46,88.55,37.22,89.52,88.72,-,-,-,-,-,-,-,-,-,-,-,-
1,0.66,36.07,0.41,0.46,88.56,37.22,37.78,37.83,37.72,89.52,87.44,-,-,-,-,-,-,-,-,-
2,87.44,89.52,99.15,93.96,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
3,88.57,88.55,37.22,87.03,89.52,93.18,93.12,93.17,87.44,93.94,88.72,99.1,99.04,99.05,93.96,-,-,-,-,-
4,0.66,36.07,0.4,0.45,37.22,88.55,89.52,-,-,-,-,-,-,-,-,-,-,-,-,-
5,0.66,0.41,88.55,37.22,89.52,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
6,0.66,88.55,89.52,37.22,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
7,0.66,36.07,0.46,0.4,88.55,37.22,89.52,-,-,-,-,-,-,-,-,-,-,-,-,-
8,0.66,36.07,0.41,0.46,88.55,37.22,93.94,89.52,-,-,-,-,-,-,-,-,-,-,-,-
9,89.52,87.44,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-


In [29]:
for x in df.filter(regex=("TIND.*")):
    i = 0
    for y in df[x]:
        a = str(search(str(y)))
        if a != 'None':
            a= ''.join([i for i in a if not i.isdigit()])
            a=  a.replace(":", "")
            dfbaru[x][i] = str(a)
            i = i+1
        else:
            dfbaru[x][i] = "-"
            i = i+1

C:\Users\Irvin\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Irvin\anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [27]:
dfbaru

,Usia,LOS,DX1,DX2,DX3,DX4,DX5,DX6,DX7,DX8,...,TIND11,TIND12,TIND13,TIND14,TIND15,TIND16,TIND17,TIND18,TIND19,TIND20
0,54,-262,ST elevation (STEMI) myocardial infarction of ...,"Atrioventricular block, first degree",-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
1,58,41,ST elevation (STEMI) myocardial infarction of ...,"Atrioventricular block, complete","Acute kidney failure, unspecified",-,Type 2 diabetes mellitus with circulatory comp...,"Obesity, unspecified",Hypo-osmolality and hyponatremia,Essential (primary) hypertension,...,"Open wound of pharynxOpen wound of pharynx, wi...",-,-,-,-,-,-,-,-,-
2,69,-262,Atrial fibrillation and flutter,Atherosclerotic heart disease of native corona...,Toxic liver disease with cholestasis,Ventricular tachycardia,Presence of cardiac pacemaker,"Disorders of plasma-protein metabolism, not el...",-,-,...,-,-,-,-,-,-,-,-,-,-
3,60,-111,"Cerebral infarction, unspecified",Type 2 diabetes mellitus without complications,Non-ST elevation (NSTEMI) myocardial infarction,Pneumonia due to other specified infectious or...,"Acute kidney failure, unspecified",Essential (primary) hypertension,Hyperuricemia without signs of inflammatory ar...,Hyperkalemia,...,"Amput abv elbow, unilatTraumatic amputation of...",Effects of reduced temperatureNone,-,-,-,-,-,-,-,-
4,65,-203,ST elevation (STEMI) myocardial infarction of ...,Essential (primary) hypertension,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
5,56,-263,Old myocardial infarction,"Angina pectoris, unspecified",-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
6,74,-263,"Angina pectoris, unspecified",Essential (primary) hypertension,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
7,58,-234,Subsequent ST elevation (STEMI) myocardial inf...,Hyperkalemia,-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
8,59,91,"Angina pectoris, unspecified",Essential (primary) hypertension,Other and unspecified asthma,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-
9,57,213,Old myocardial infarction,"Acute kidney failure, unspecified",-,-,-,-,-,-,...,-,-,-,-,-,-,-,-,-,-


In [30]:
dfbaru.to_csv('Clean.csv')

In [34]:
dfbaru.to_excel('Clean.xlsx',index= False)